<div style="text-align:center;">
  <img src="https://github.com/MolSSI-Education/iqb-2025/blob/main/images/molssi_main_outline.png?raw=true" style="display: block; margin: 0 auto; max-height:325px;">
</div>

# Exploring Chemical and Biological Data With BidingDB and the RDKit

This tutorial was written by [Pat Walters](https://patwalters.github.io) for the [Cheminformatic-Driven Molecular Docking Workshop](https://pdb101.rcsb.org/news/67d9853eaddf75595bd158f7) held as a Crash Course with the Institute for Quantitative Biomedicine (IQB) and the Protein Data Bank (PDB).

*This notebook is Part 2 of 4 in the notebook series.*

Other notebooks in this series:
1. [Digital Representation of Molecules](https://colab.research.google.com/github/MolSSI-Education/iqb-2025/blob/main/01_Cheminfo_crash_course.ipynb)
2. **Exploring Chemical and Biological Data with BindingDB and the RDKit** (This notebook)
3. [Preparing Structures for Docking](https://colab.research.google.com/github/MolSSI-Education/iqb-2025/blob/main/03_Cheminfo_crash_course.ipynb)
4. [Molecular Docking using gnina](https://colab.research.google.com/github/MolSSI-Education/iqb-2025/blob/main/04_Cheminfo_crash_course.ipynb)

In this notebook we will use the [BindingDB](https://bindingdb.org/rwd/bind/index.jsp), database of chemical structures and biological activity values extracted from the chemical literature, to search for compounds that bind to the [SARS-CoV-2 main  protease](https://pubmed.ncbi.nlm.nih.gov/35845352/).  In addition to hosting data from the scientifiic literature, BindingDB also has data extracted from chemical patents.  We will extract the data from a SARS-CoV-2 inhibitor patent, explore this data, and save a representative subset for a subsequent docking analysis.

### 1. Searching BindingDB and Downloading Chemical Structures and Data
We begin by accessing the BindingDB website at [https://bindingdb.org](https://bindingdb.org). Click on the **Publications** link on the left, then click on the **US Patent** link.
   
<img src="https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/refs/heads/main/pdb/binding_db_01.png"/>     
   
This will bring up a results page like the one below.  This page shows recently curated patents and allows us to search by patent number.  Enter this text into the box, [US20240293380](https://patents.google.com/patent/US20240293380A1/fr).  Once you click on the **Search** button, BindingDB will find compounds that were curated from this patent by scientists as the Cleveland Clinic, describing a set of protease inhibitors.
   
<img src="https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/refs/heads/main/pdb/binding_db_02.png"/>  
   
This will take you to a page with details the 75 compounds extracted from US20240293380.  The page shows chemical structures as well as links to other databases. To download the information about all structures from this patent, first click on the **Add all pages** link, followed by the **Make dataset** button indicated by the arrows in figure below.
   
<img src="https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/refs/heads/main/pdb/binding_db_03.png"/>   
   
By selecting the **Tab Delimited (TSV)** radio button and pressing the **Go** button, we can download the data.  This download requires registration with BindingDB.  To streamline this tutorial, we will load the file directly from GitHub instead.
   
<img src="https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/refs/heads/main/pdb/binding_db_04.png"/>    

### 2. Install and Import the necessary Python libraries
The next cell will install the Python libraries necessary for this notebook.  Note that the libraries are only installed if the notebook is running in Google Colab.  The installation process generates a lot of output, which we suppress with the **%%caputure%%** magic command.  If you run into problems, remove **%%capture%%** to see the cell output.

In [ ]:
%%capture
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install pandas mols2grid seaborn rdkit matplotlib useful_rdkit_utils scikit-learn ipywidgets

We are the capabilities provided by these libraries.

- [pandas](https://pandas.pydata.org/) - data analysis and manipulation
- [mols2grid](https://github.com/cbouy/mols2grid) - display interactive grids of chemical structures and data
- [seaborn](https://seaborn.pydata.org/) - plotting and data visualization
- [rdkit](https://www.rdkit.org/) - Open source cheminformatics
- [matplotlib](https://matplotlib.org/) - data visualization
- [useful_rdkit_utils](https://useful-rdkit-utils.readthedocs.io/en/latest/) - Cheminformatics utilities
- [scikit-learn](https://scikit-learn.org/stable/) - Machine learning
- [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/) - Interactive tools in Jupyter notebooks

In the cell below, we load the python libraries

In [ ]:
import pandas as pd
import mols2grid
import seaborn as sns
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
import useful_rdkit_utils as uru
from IPython.core.display import HTML

### 3. Read the file we downloaded from PDBBind into a Pandas dataframe.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/refs/heads/main/pdb/monomers_12285.tsv",sep="\t")

As we can see, the data we extracted from BindingDB has many fields.

In [ ]:
df.columns

To make things simpler, we'll rename a couple of columns.

In [ ]:
df.rename(columns={'Ligand SMILES':'SMILES'},inplace=True)
df.rename(columns={'BindingDB Ligand Name':'Name'},inplace=True)

We can aslo see that the data has 75 rows, with each compound corresponding to a row.  For our purposes, we'll focus on a few fields.
- SMILES - the chemical structure of the compound
- Name - the internal identifier Binding DB provides for each compund
- IC50 (nM) - the IC50 in nM.

The IC50, or half maximal inhibitory concentration, is a measure of a substance's potency in inhibiting a specific biological or biochemical function. It represents the concentration of an inhibitor needed to reduce a particular biological process or component by 50% in vitro, and is commonly used to assess the effectiveness of drugs in pharmacological research. IC50 values are typically expressed as nM concentrations and provide a quantitative indication of how much of an inhibitory substance is required to achieve half-maximal inhibition of a given target or process

In [ ]:
len(df)

We can use [mols2grid](https://github.com/cbouy/mols2grid) to browse the chemcial structures of the 75 compounds.

In [ ]:
mols2grid.display(df,smiles_col="SMILES",subset=["img","IC50 (nM)"])

### Exercise #1
1. Explort the SMILES, Name, and IC50 fields to a csv <br/>
2. Read that into a dataframe called **my_df**
3. Sort **my_df** by the IC50 column

### 3. Visualizing the Activity Distribution
First, let's set some reasonable defaults for plots.

In [ ]:
sns.set(rc={'figure.figsize': (5,5 )})
sns.set_style('whitegrid')
sns.set_context('notebook')

Let's look at the the IC50 distribution for the 75 compounds.  As we can see below, the default view isn't very helpful.

In [ ]:
ax = sns.histplot(df["IC50 (nM)"])

It's oftent easier to look at the distribution of **pIC50** values, where pIC50 is the negative log of the IC50.

In [ ]:
df["pIC50"] = -np.log10(df["IC50 (nM)"] * 1e-9)

In [ ]:
ax = sns.histplot(df.pIC50)

### Exercise #2
Find the maximum, minimum, and mean(average) pIC50 values


For visualization it's often helpful to bin the activity into high, medium, and low bins.  Here well put compounds with 6 < pIC50 < 7 (0.1-1$\micro$M) into the "medium" bin, and compounds with pIC50 > 7 into the "high" bin.

In [ ]:
df["activity"] = pd.cut(df.pIC50,bins=[0,6,7,100],labels=["low","med","high"],right=False)

In [ ]:
ax = sns.histplot(x="pIC50",hue="activity",data=df,bins=np.arange(4,9,0.5))

Now that we have the IC50 data binned into high, med, low, we can use these categories to make plots showing mulitple parameters.  In the cells below we'll make a scatterplot of molecular weight (MW) vs LogP, a measure of lipophilicity. We can then color the plot by activty and see whether acitivity is driven by MW or LogP.

To calculate MW and LogP we need to add an RDKit molecule column to the dataframe.  As we will see in next couple of cells, the Pandas [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method provides a handy way of performing operations on an entire dataframe column.

In [ ]:
df['mol'] = df.SMILES.apply(Chem.MolFromSmiles)

Now that the dataframe has a **mol** column with the RDKit molecules, we can use **apply** to calculate MW and LogP.

In [ ]:
df['MW'] = df.mol.apply(uru.MolWt)
df['LogP'] = df.mol.apply(uru.MolLogP)

Use [seaborn](https://seaborn.pydata.org/) to make the scatterplot of MW vs LogP, colored by **activity**. Note that while we don't have a clear relationship between either MW or LogP and activity, the smallest molecules tend to have low to medium activity.

In [ ]:
ax = sns.scatterplot(x="MW",y="LogP",hue="activity",data=df)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

### Exercise #3
Make a scatterplot of the number of hydrogen bond donors vs the number of hydrogen bond acceptors for the dataset.<br>
**Hint**: the functions you need to use are uru.NumHDonors and uru.NumHAcceptors

### 4. Understanding the Chemistry in the Downloaded Molecules
We downloaded 75 molecules from BindingDB.  It can often be challenging to understand the trends in sets of molecules. In the next section, we'll look at how Cheminformatics tools can aid or exploratory data analysis. As a first step in this analysis, we will see which parts are common to all 75 of the molecules.  To do this, we can use the Maxiumum Common Substructure (MCS) capability in the RDKit.  To use this capability, we must first convert the SMILES in our dataframe to RDKit molecule objects.  As above, we can use the Pandas apply method to do this.

In [ ]:
df['mol'] = df["SMILES"].apply(Chem.MolFromSmiles)

Next we can calculate the MCS.

In [ ]:
mcs = rdFMCS.FindMCS(df.mol)

To visualize the MCS, we have to convert it to an RDKit molecule object.

In [ ]:
pat = Chem.MolFromSmarts(mcs.smartsString)
pat

The visualization above can be difficult to interpret.  The MCS tends to make more sense when we map it onto the molecules. Here we can map the MCS on to the first molecule in the dataframe and display it as a red highlight. Note how we use the utilitiy function [rd_set_image_size](https://useful-rdkit-utils.readthedocs.io/en/latest/jupyter.html#useful_rdkit_utils.jupyter_utils.rd_set_image_size) from [useful_rdkit_utils](https://useful-rdkit-utils.readthedocs.io/en/latest/index.html) to set the image size.

In [ ]:
uru.rd_set_image_size(300,300)
df.mol.values[0].GetSubstructMatch(pat)
df.mol.values[0]

Another tool for exploring chemical datasets is the Bemis-Murcko scaffold. In this technique a molecule is reduced to a set of connected rings and linkers.  We can use the **GetScaffoldForMol**  method from the RDKit to generate Bemis-Murcko scaffolds.  

In [ ]:
df['Scaffold'] = df.mol.apply(GetScaffoldForMol)

The generated scaffolds are RDKit molecule objects.  We would like to find the most frequently occuring scaffolds.  To do this, we need to convert the scaffolds to SMILES.  

In [ ]:
df['Scaffold_SMILES'] = df.Scaffold.apply(Chem.MolToSmiles)

Now we can use the [value_counts_df](https://useful-rdkit-utils.readthedocs.io/en/latest/pandas.html#useful_rdkit_utils.pandas_utils.value_counts_df) method in [useful_rdkit_utils](https://useful-rdkit-utils.readthedocs.io/en/latest/index.html) count how many times each scaffold is used.  

In [ ]:
scaffold_counts_df = uru.value_counts_df(df,"Scaffold_SMILES")
scaffold_counts_df.head()

Use the **scaffold_counts_df** we created above to view the scaffolds and their frequencies.

In [ ]:
mols2grid.display(scaffold_counts_df,smiles_col="Scaffold_SMILES",subset=["img","count"],size=(250,250),
                  n_items_per_page=6)

Let's take a closer look at the most frequent scaffolds.  We can use the Pandas **query** method to find scaffolds that occur in at least 5 molecules.  

In [ ]:
uru.value_counts_df(df,"Scaffold_SMILES").query("count > 5")

### Exercise #4
Visualize the scaffolds that occur fewer than 5 times.

In the next section, we'll look at how we can plot the activity distribution associated with each of the scaffolds specified above.  We will use the Pandas groupby function to select compounds with the same scaffold.  Once we've grouped by scaffold,  we can compare the corresponding IC50 distributions.

In [ ]:
output_list = []
for k,v in df.groupby('Scaffold_SMILES'):
    if len(v) > 5:
        mol = Chem.MolFromSmiles(k)
        mol_image = uru.mol_to_base64_image(mol)
        boxplot_image = uru.boxplot_base64_image(v.pIC50.values)
        output_list.append([mol_image, len(v),boxplot_image])
output_df = pd.DataFrame(output_list,columns=["Scaffold","Num Examples","pIC50 Distribution"])

With the dataframe generated above, we can now visualize the activity distributions associated with the three most common scaffolds.

In [ ]:
HTML(output_df.sort_values("Num Examples",ascending=False).to_html(escape=False))

### 5. Select and Export a Subset of the 75 Compounds for Docking
As the next part of this tutorial, we'll dock a few of the 75 molecules we extracted into a protein crystal structure of p38 kinase. Since we have limited commputational resources available, we will use clustering to select a subset of the compounds.   Clustering uses chemical fingerprits to group similar chemical structures.  In this case, we will use the Butina clustering algorithm as implemented in the RDKit.  To perform the clustering, we need to generate a different chemical fingerprint format than the one we generated above when we were performing dimensionality reduction. In this case, we will use the **get_fp** method of the useful_rdkit_utils Smi2Fp class.  

In [ ]:
smi2fp = uru.Smi2Fp()
df['morgan_fp'] = df.SMILES.apply(smi2fp.get_fp)

With the fingerprint generated, we can cluster using the **taylor_butina_clustering** convenience function in useful_rdkit_utils. This adds a new field, "cluster" to the dataframe.

In [ ]:
df['cluster'] = uru.taylor_butina_clustering(df.morgan_fp)

As we did above, we can use the Pandas **value_counts** method to see how many compounds are in each cluster. The vast majority of compounds are in cluster 0.

In [ ]:
df.cluster.value_counts()

Let's select the most active compound from each cluster.  To do this we first sort the dataframe by "IC50 (nM)".  Once the dataframe is sorted, we can use the Pandas **drop_duplicates** methood to delete all but the first compound from each cluster.

In [ ]:
unique_cluster_df = df.sort_values("IC50 (nM)").drop_duplicates("cluster").copy()

Now we'd like to exort these 5 compounds as a csv file.  The dataframe contains a lot of columns, most of which we don't need.  Let's creat a list of the columns we'd like to export.

In [ ]:
columns_to_keep = ["SMILES","Name","IC50 (nM)","cluster"]

View these columns to ensure this is what we want.

In [ ]:
unique_cluster_df[columns_to_keep].sort_values("cluster")

Finally, export the five molecules as a csv file.

In [ ]:
unique_cluster_df[columns_to_keep].sort_values("cluster").to_csv("US20240293380_examples.csv",index=False)

These structures can be docked into the PDB structure [7LME](https://www.rcsb.org/structure/7lme) of the SARS-CoV-2 3CLPro. For more information on compounds similar to those in US20240293380, please see ["Structure-Based Optimization of ML300-Derived, Noncovalent Inhibitors Targeting the Severe Acute Respiratory Syndrome Coronavirus 3CL Protease (SARS-CoV-2 3CLpro)"](https://pubs.acs.org/doi/10.1021/acs.jmedchem.1c00598)

## Exercise #5
Create a dataframe with the least active molecule from each cluster.